# Visualize Channel Toggle Bg

Notebook to plot the selected channel for a list of image files.
It can be used for instance, to plot all the steps of the low-level cleaning for a specific channel for 1 point, or compare the expression of a specific channel across different points.
This version of the script takes 2 images, plots them in the same axis object and allows the user to toggle between both of them: once the notebook has run, select the desired plot and press 't' to toggle the visibility of both plots. This is useful for instance, in order to visualize the change in a channel before and afer slide bg removal.

Accepted format for the images: IONpath's **MIBItiff** as from the MIBI/O software or the MIBItracker.

Please update the parameters in the **input parameters** section before running the notebook.

# load headers

In [ ]:
# magic commands
%matplotlib notebook
%load_ext autoreload
%autoreload 2

#imports
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mibidata import mibi_image as mi, tiff
import visualize_data as viz

# input parameters

In [ ]:
data_path = '../sample_data'

# channel to plot
# define as mass, in order to anonymize the targets
channel_to_plot = 115

slide_num = 1
point_num = 1
run_name = 'CohortSlide1'

# load tiffs
l_tiff_files = []
l_tiff_files_bg = []
l_titles = []

l_tiff_files.append(os.path.join(data_path,
                                 f'{run_name}/{run_name}_TIFF',
                                 'bg_au_050_ta_020',
                                 f'Point{point_num}_RowNumber0_Depth_Profile0.tiff'))
l_tiff_files_bg.append(os.path.join(data_path,
                                    f'{run_name}/{run_name}_TIFF',
                                    'bg_none',
                                    f'Point{point_num}_RowNumber0_Depth_Profile0.tiff'))
l_titles.append('bg removed: Au 50, Ta 20')

l_tiff_files.append(os.path.join(data_path,
                                 f'{run_name}/{run_name}_TIFF',
                                 'bg_au_050_ta_020',
                                 f'Point{point_num}_RowNumber0_Depth_Profile0-MassCorrected.tiff'))
l_tiff_files_bg.append(os.path.join(data_path,
                                    f'{run_name}/{run_name}_TIFF',
                                    'bg_au_050_ta_020',
                                    f'Point{point_num}_RowNumber0_Depth_Profile0.tiff'))
l_titles.append('isobaric interferences corrected')

l_tiff_files.append(os.path.join(data_path,
                                 f'{run_name}/{run_name}_TIFF',
                                 'bg_au_050_ta_020',
                                 f'Point{point_num}_RowNumber0_Depth_Profile0-MassCorrected-Filtered.tiff'))
l_tiff_files_bg.append(os.path.join(data_path,
                                    f'{run_name}/{run_name}_TIFF',
                                    'bg_au_050_ta_020',
                                    f'Point{point_num}_RowNumber0_Depth_Profile0-MassCorrected.tiff'))
l_titles.append('denoised')

#anonymize_targets = True
anonymize_targets = False

if anonymize_targets:
    print('WARNING! TIFF files and TIFF file names might not be anonymized!')

# plot images

In [ ]:
# dark plot style
plt.style.use('dark_background')

# loop over files
sort_dict = {}
for f_order, (tiff_file, tiff_file_bg, title) in enumerate(zip(l_tiff_files, l_tiff_files_bg, l_titles)):
    print()
    if anonymize_targets:
        print(f'file {f_order}: \'{title}\', channel {channel_to_plot}')
    else:
        print(f'file {f_order}: {tiff_file}, channel {channel_to_plot}')
        print(title)
    #if f_order < 5: # debug
    # read MIBItiff image file
    image = tiff.read(tiff_file)
    # read MIBItiff bg image file
    image_bg = tiff.read(tiff_file_bg)
    #if not anonymize_targets:
        #print(f'metadata {image.metadata}')
        #print(f'channels {image.channels}')
    im = image[channel_to_plot]
    im_bg = image_bg[channel_to_plot]
    counts = im.sum()
    
    #sorting metric (integer value initialized as float to deal 'same-key' issue)
    metric = float(np.abs(im - im_bg).sum())
    # online sort in dictionary (<100 channels so we can add .01 to deal with same-metric channels)
    while metric in sort_dict:
        metric += 0.01
    sort_dict[metric] = (im, im_bg, channel_to_plot, counts, title)
    
    #im_axes.append(viz.plot_toggle_image(im, im_bg,
    #                                     ax=None,
    #                                     #title=str(f_order) + ' ' + str(channel_to_plot) + ' ' + str("{:.2e}".format(counts)),
    #                                     title=str(channel_to_plot) + ' ' + str("{:.2e}".format(counts)) + ': \'' + title + '\'',
    #                                     #brighten_image=True, hi_res=True))
    #                                     brighten_image=True, hi_res=False))
    
    #break # debug (show only 1 plot)
# sort and plot images
im_axes = []
for met in sorted(sort_dict.keys(),reverse=True):
    im_axes.append(viz.plot_toggle_image(sort_dict[met][0], sort_dict[met][1],
                                         ax=None,
                                         #title=str(f_order) + ' ' + str(channel_to_plot) + ' ' + str("{:.2e}".format(counts)),
                                         title=str(sort_dict[met][2]) + ' ' + str("{:.2e}".format(sort_dict[met][3])) + ': \'' + sort_dict[met][4] + '\'',
                                         #brighten_image=True, hi_res=True))
                                         brighten_image=True, hi_res=False))
    #break # debug (show only 1 plot)